In [1]:
import math
import numpy as np
import gurobipy as gp
from gurobipy import GRB, quicksum as qsum

In [30]:
k = 6  # k - номер студента по списку
l = 1  # l - номер группы

n = 10 + math.floor(k/4.0)
m = 30 - math.floor(k/4.0)

A = np.array([
        np.array([
            -k + ((1451*i + 1571*j + 2081*k + 2543*l) % (30 + math.floor(k/5.0)))
            for j in range(1, n + 1)
        ])
        for i in range(1, m + 1)
    ])

A = np.array(A)

In [7]:
A.shape

(29, 11)

In [99]:
minimum = A[0][0]
for i in range(A.shape[0]):
    for j in range(A.shape[1]):
        if A[i][j] < minimum:
            minimum = A[i][j]
print('Минимальный элемент:', minimum)
alpha = 0
if minimum <= 0:
    alpha = -minimum + 1
print('Технологическая матрица:')
print(A + alpha)

Минимальный элемент: -6
Технологическая матрица:
[[10 31 21 11  1 22 12  2 23 13  3]
 [ 4 25 15  5 26 16  6 27 17  7 28]
 [29 19  9 30 20 10 31 21 11  1 22]
 [23 13  3 24 14  4 25 15  5 26 16]
 [17  7 28 18  8 29 19  9 30 20 10]
 [11  1 22 12  2 23 13  3 24 14  4]
 [ 5 26 16  6 27 17  7 28 18  8 29]
 [30 20 10 31 21 11  1 22 12  2 23]
 [24 14  4 25 15  5 26 16  6 27 17]
 [18  8 29 19  9 30 20 10 31 21 11]
 [12  2 23 13  3 24 14  4 25 15  5]
 [ 6 27 17  7 28 18  8 29 19  9 30]
 [31 21 11  1 22 12  2 23 13  3 24]
 [25 15  5 26 16  6 27 17  7 28 18]
 [19  9 30 20 10 31 21 11  1 22 12]
 [13  3 24 14  4 25 15  5 26 16  6]
 [ 7 28 18  8 29 19  9 30 20 10 31]
 [ 1 22 12  2 23 13  3 24 14  4 25]
 [26 16  6 27 17  7 28 18  8 29 19]
 [20 10 31 21 11  1 22 12  2 23 13]
 [14  4 25 15  5 26 16  6 27 17  7]
 [ 8 29 19  9 30 20 10 31 21 11  1]
 [ 2 23 13  3 24 14  4 25 15  5 26]
 [27 17  7 28 18  8 29 19  9 30 20]
 [21 11  1 22 12  2 23 13  3 24 14]
 [15  5 26 16  6 27 17  7 28 18  8]
 [ 9 30 20 10 3

In [94]:
def runner(A):
#     A = A.T
    m = len(A)
    n = len(A[0])
    b = np.array([1 for _ in range(m)])
    c = np.array([1 for _ in range(n)])
    
    
    # решение в чистых стратегиях
    minmax = min([max(A.T[i]) for i in range(n)])
    maxmin = max([min(A[i]) for i in range(m)])
    print('minmax:', minmax)
    print('maxmin:', maxmin)
    
    if minmax == maxmin:
        print('Решение в чистых стратегиях: ')
        for i in range(n):
            if max(A.T[i]) == minmax:
                print('i =', i)
                break
        for j in range(m):
            if max(A[j]) == maxmin:
                print('j =', j)
                break
        return
    else:
        print('Решений в чистых стратегий нет')
    
    
    # все элементы матрицы делаем положительными
    minimum = A[0][0]
    for i in range(A.shape[0]):
        for j in range(A.shape[1]):
            if A[i][j] < minimum:
                minimum = A[i][j]
    alpha = 0
    if minimum <= 0:
        alpha = -minimum + 1
    A = A + alpha
    
    
    model = gp.Model()
    
    # задание переменных
    model.addMVar(len(c), lb=[0.0 for _ in range(len(c))], vtype=GRB.CONTINUOUS, name='x')
    model.update()
    x = gp.MVar(model.getVars())
    
    # задание оптимизационного критерия
    model.setObjective(c.T @ x, GRB.MINIMIZE)
    
    model.addConstr(A @ x >= b)
    
    # оптимизация
    model.optimize()

    # вывод результатов
    x_star = []
    for v in model.getVars():
        x_star += [v.x]
    v_star = 1.0 / model.ObjVal
    v_star_real = v_star - alpha
    print('alpha: ' + str(alpha))
    print('v_star: ' + str(v_star_real))
    x_normal = [v_star*x for x in x_star]
    print('x_star: ' + str(x_normal))
    print('sum of x_star: ' + str(sum(x_normal)))

In [95]:
runner(A)

minmax: 24
maxmin: 1
Решений в чистых стратегий нет
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 29 rows, 11 columns and 319 nonzeros
Model fingerprint: 0xf886c3a7
Coefficient statistics:
  Matrix range     [1e+00, 3e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve time: 0.01s
Presolved: 29 rows, 11 columns, 319 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   3.625000e+00   0.000000e+00      0s
      11    7.3985680e-02   0.000000e+00   0.000000e+00      0s

Solved in 11 iterations and 0.02 seconds (0.00 work units)
Optimal objective  7.398568019e-02
alpha: 7
v_star: 6.516129032258066
x_star: [0.03225806451612902, 0.032258064516129045, 0.03225806451612886, 0.0322580645161292, 0.03225806451612901, 0.03225806451612904, 0.03225806451612899, 0.03225806451612901, 0.

In [96]:
runner(-A)

minmax: 6
maxmin: -17
Решений в чистых стратегий нет
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 29 rows, 11 columns and 319 nonzeros
Model fingerprint: 0x67f165b6
Coefficient statistics:
  Matrix range     [1e+00, 3e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve time: 0.01s
Presolved: 29 rows, 11 columns, 319 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   3.625000e+00   0.000000e+00      0s
      11    7.3985680e-02   0.000000e+00   0.000000e+00      0s

Solved in 11 iterations and 0.01 seconds (0.00 work units)
Optimal objective  7.398568019e-02
alpha: 25
v_star: -11.483870967741932
x_star: [0.22580645161290322, 0.2580645161290323, 0.2580645161290323, 0.03225806451612904, 0.03225806451612903, 0.03225806451612905, 0.032258064516129, 0.032258064516129045, 0

In [97]:
runner(np.dot(A.T, A))

minmax: 4498
maxmin: 1443
Решений в чистых стратегий нет
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 11 rows, 11 columns and 121 nonzeros
Model fingerprint: 0xdb05c25c
Coefficient statistics:
  Matrix range     [1e+03, 5e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve time: 0.01s
Presolved: 11 rows, 11 columns, 121 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   1.718750e-01   0.000000e+00      0s
      11    4.0649505e-04   0.000000e+00   0.000000e+00      0s

Solved in 11 iterations and 0.02 seconds (0.00 work units)
Optimal objective  4.064950482e-04
alpha: 0
v_star: 2460.0545677498467
x_star: [0.12903225806451613, 0.09946401376555074, 0.11789719348905245, 0.03225806451612907, 0.032258064516129045, 0.032258064516129774, 0.03225806451612897, 0.0322580645161